In [27]:
import os
from fastkml import kml
import zipfile

# use existing kmz ZipFile if present, otherwise open the file path
kmz_file = "L:/TO_Traffic/TMC/000 - TMC References/Google Earth Files/IntersectionControl_080421.kmz"
extraction_dir  = os.path.dirname("./maps")
with zipfile.ZipFile(kmz_file, "r") as kmz:
    kmz.extractall(extraction_dir)
import fiona
import geopandas as gpd
import pandas as pd
fp_eq = os.path.join(extraction_dir, "doc.kml")
gdf_list = []
for layer in fiona.listlayers(fp_eq) :    
    gdf = gpd.read_file(fp_eq, driver='LIBKML', layer=layer)
    gdf_list.append(gdf)

gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
gdf["long"] = gdf.geometry.x
gdf["lat"] = gdf.geometry.y
gdf.drop(columns=["Description", "geometry"], inplace=True)
gdf.to_csv("intersections.csv", index=False)

In [28]:
from glob import glob
import os
def get_files(path):
    # Get the files and not the folders absolute paths.
    # if folder, do get_files recursively
    for existing in glob(os.path.join(path, "*")):
        if os.path.isdir(existing):
            yield from get_files(existing)
        else:
            yield existing


TMC_folder_path = "L:/TO_Traffic/TMC"
all_files = []
for file in get_files(TMC_folder_path):
    all_files.append(file)


In [29]:
import json
with open("all_tmc_files.json", "w") as f:
    json.dump(all_files, f, indent=4)